# Taproot BIP 322 Signing

This notebook experiments with using taproot based addresses for BIP 322 signing

## Types of Taproot Scripts to Test

See Jimmy Song presentation on [Taproot MultiSig](https://jimmysong.github.io/taproot-multisig/)

1. Schnorr pay to public key
2. Schnorr MuSig signer (n of n)
3. Single leaf multi sig TapScript
4. Multi leaf k-of-k TapScript
5. Multi leaf MuSig TapScript
6. Multi leaf everything TapScript

Initially, this notebook will just focus creating and verify BIP 322 signatures using types 1 and 2. Although the presentation referenced above details why the other types may be interesting to support.

In [1]:
cd ..

/home/will/work/LegendaryRequirements/clients/dcd/bip0322-sigs


## Schnorr pay to public key

This just uses a taproot based address which can be unlocked with a single schnorr signature from a specific private key.

### Import PrivateKey

In [2]:
from buidl.ecc import PrivateKey

In [3]:
private_key_wif = "L3VFeEujGtevx9w18HD1fhRbCH67Az2dpCymeRE1SoPK6XQtaN2k"

In [4]:
private_key = PrivateKey.parse(private_key_wif)

### Define Message to Sign

In [5]:
message = "Hello World"

### Generate A Pay to Tap Root Script for the Secp256k1 Point associated with The Private Key

In [6]:
p2tr_script = private_key.point.p2tr_script()
print(p2tr_script)

OP_1 0b34f2cc6f60d54e3fdc2d1dd053fcc393bd2db9acc8de4a7c3cc28a83d4d8e9 


### Convert Script to Bitcoin Address Representation

In [7]:
p2tr_address = p2tr_script.address()
print(p2tr_address)

bc1ppv609nr0vr25u07u95waq5lucwfm6tde4nydujnu8npg4q75mr5sxq8lt3


### Calculate Teaked Private Key for the P2TR Address

See the Tweaked Keys section :  https://medium.com/coinmonks/on-bitcoins-schnorr-signature-algorithm-and-taproot-script-and-witness-sizes-fe4d1e6591a7

In [8]:
from buidl.taproot import TapRoot
taproot = TapRoot(private_key.point)
tweaked_private_key = private_key.tweaked(taproot.tweak)

In [9]:
taproot

### Sign Message using Tweaked Private Key and Taproot Address

In [10]:
from src.message import sign_message, MessageSignatureFormat
signature = sign_message(MessageSignatureFormat.SIMPLE, tweaked_private_key, p2tr_address, message)
print("BIP 322 p2tr based Signature: ", signature)

BIP 322 p2tr based Signature:  AUHd69PrJQEv+oKTfZ8l+WROBHuy9HKrbFCJu7U1iK2iiEy1vMU5EfMtjc+VSHM7aU0SDbak5IUZRVno2P5mjSafAQ==


### Verify Signature against p2tr_address and message

In [19]:
from src.message import verify_message

is_valid = verify_message(p2tr_address, signature, "HelloWorldasd")

print("BIP 322 Signature \n" + signature + "\nagainst message : " + message + " and address : " + p2tr_address + "\nis valid? ", is_valid)

mismatch between length and consumed bytes 76 vs 25
BIP 322 Signature 
AUHd69PrJQEv+oKTfZ8l+WROBHuy9HKrbFCJu7U1iK2iiEy1vMU5EfMtjc+VSHM7aU0SDbak5IUZRVno2P5mjSafAQ==
against message : Hello World and address : bc1ppv609nr0vr25u07u95waq5lucwfm6tde4nydujnu8npg4q75mr5sxq8lt3
is valid?  False


In [12]:
is_valid

True

## Schnorr MuSig Signing

This is a p2tr address that requires a n-of-n Schnorr aggregated signature.

**Note: The current API defined in message.py cannot support MuSig. Or other more complex Bitcoin Script. Need something along the lines of Partially Signed BIP 322 Signature**

**Note2: PSBT in buidl-python does not to support taproot yet (See open issue https://github.com/buidl-bitcoin/buidl-python/issues/107)**

In [13]:
secret1 = 123817492183
priv_key1 = PrivateKey(secret=secret1)

secret2 = 7529884329432
priv_key2 = PrivateKey(secret=secret2)

In [14]:
aggregated_pubkey = priv_key1.point + priv_key2.point

In [15]:
taproot = TapRoot(aggregated_pubkey)
taproot.tweak

9288271874445777610288052398967688125321604121694257413205830807510323674645

In [16]:
musig_taproot_address = taproot.address()

In [17]:
tweaked_pk_1 = priv_key1.tweaked(taproot.tweak)

In [18]:
signature = sign_message(MessageSignatureFormat.FULL,tweaked_pk_1,musig_taproot_address,message)

RuntimeError: Unable to sign message